In [1]:
import pandas as pd
import re
from tqdm import tqdm
import os
from konlpy.tag import Mecab
from tensorflow.keras.preprocessing import sequence
from transformers import BertTokenizer
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from transformers import TFElectraModel
from sklearn.metrics import classification_report
import numpy as np

import pandas as pd
from tqdm import tqdm

import argparse
import time
import gc
import numpy as np
import collections
import sys
import random
import openpyxl

In [2]:
class TFElectraClassifier(tf.keras.Model):
    def __init__(self, num_class):
        super(TFElectraClassifier, self).__init__()

        self.model = TFElectraModel.from_pretrained('../nscc_mecab_base_final/torch_model', from_pt=True)

        #self.electra_layers = model.get_layer('electra')
        
        self.dropout = tf.keras.layers.Dropout(0.1)
        self.classifier = tf.keras.layers.Dense(num_class, 
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02), 
                                                name="classifier", activation='softmax')
        
    def call(self, inputs, attention_mask=None, token_type_ids=None, training=False):
        
        #outputs 값: # sequence_output, pooled_output, (hidden_states), (attentions)
        outputs = self.model(inputs, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = keras.layers.Lambda(lambda seq: seq[:, 0, :])(outputs[0])
        pooled_output = self.dropout(pooled_output, training=training)
        logits = self.classifier(pooled_output)

        return logits

In [3]:
desc = 'keyword_classification_22.3.9-2'
lr = 5e-05

In [4]:
uniq_keywords = pd.read_csv('uniq_keywords, codes, uniq_token_keywords_220306.csv')

In [5]:
uniq_keywords

,Unnamed: 0,keyword,code,token_keyword,encode_keyword
0,0,강관 접합부 용접 볼트 편측,EC01,강관 접합부 용접 볼트 편 ##측,"[202, 16793, 20616, 10051, 10970, 5796, 6463, ..."
1,1,유연신축성 생체신호 감지 유기 센서,EC02,유연 신축성 생체 ##신호 감지 유기 센서,"[202, 11967, 25171, 11028, 9400, 9288, 9240, 9..."
2,2,크론병궤양성 대장염clusterin,LC02,크 ##론 ##병 궤 ##양성 대장 ##염 cl ##ust ##er ##in,"[202, 5668, 6638, 6722, 4223, 27793, 15805, 63..."
3,3,bulk heterojunction,EF06,b ##ul ##k he ##ter ##o ##j ##unction,"[202, 270, 9643, 6343, 18616, 9910, 6433, 6408..."
4,4,글로벌 리더 양성바이오헬스케어 소재,LA09,글로벌 리더 양성 바이오 ##헬 ##스케 ##어 소재,"[202, 18621, 14452, 14816, 11662, 7337, 17853,..."
...,...,...,...,...,...
219451,219451,산통,LB07,산 ##통,"[202, 4975, 6584, 203]"
219452,219452,휴폭,LB01,휴 ##폭,"[202, 5937, 6762, 203]"
219453,219453,본칩,LA08,본 ##칩,"[202, 4878, 6860, 203]"
219454,219454,근로,EE02,근로,"[202, 29274, 203]"


In [6]:
keywords = uniq_keywords['keyword']
codes = uniq_keywords['code']

In [7]:
regex = re.compile('[^ 가-힣a-zA-Z!@#$%^&*,]|[가나다라마바사]\)|[0-9]-?[0-9]?\)')
mecab = Mecab()


f = open('../model/labels.txt', 'r')
labels = []
while True:
    line = f.readline().replace('\n', '')
    if not line:
        break
    labels.append(line)
f.close()

In [8]:
categories = {}
for i, l in enumerate(labels):
    categories[l] = i
print("Train Labels : {}".format(len(categories)))

sample_count = {}
x_data = []
y_data = []
for n, (l, d) in enumerate(zip(codes, keywords)):
    if l[:4] not in categories:
        continue
    else:
        tmp = ''
        for s in d:
            if s == '\n':
                tmp = tmp + " [SEP] "
            else:
                tmp = tmp + s
        x_data.append(tmp)
        y_data.append(categories[l[:4]])
        
str_train_x, str_test_x, train_y, test_y = train_test_split(x_data,y_data, test_size=0.3, random_state=72)

print(f"Train : {len(str_train_x)} / Test : {len(str_test_x)}")

# print(f"Train : {len(x_data)} / Test : {len(y_data)}")

Train Labels : 145
Train : 153619 / Test : 65837


In [9]:
len(train_y)
len(test_y)

65837

In [10]:
x_data
y_data
# len(x_data)

[21,
 22,
 108,
 58,
 88,
 40,
 16,
 109,
 80,
 50,
 103,
 122,
 22,
 144,
 82,
 43,
 87,
 109,
 132,
 131,
 81,
 16,
 38,
 109,
 108,
 109,
 108,
 109,
 35,
 125,
 119,
 89,
 107,
 41,
 61,
 41,
 45,
 8,
 32,
 33,
 58,
 72,
 58,
 80,
 30,
 128,
 1,
 31,
 104,
 115,
 114,
 41,
 62,
 90,
 85,
 45,
 129,
 46,
 143,
 129,
 108,
 107,
 8,
 114,
 1,
 24,
 43,
 58,
 45,
 1,
 107,
 8,
 38,
 29,
 86,
 121,
 126,
 41,
 108,
 6,
 15,
 132,
 131,
 51,
 1,
 109,
 81,
 32,
 24,
 85,
 125,
 108,
 144,
 113,
 108,
 46,
 109,
 127,
 109,
 108,
 103,
 35,
 34,
 2,
 39,
 42,
 115,
 85,
 57,
 82,
 83,
 107,
 115,
 107,
 107,
 113,
 85,
 103,
 58,
 130,
 80,
 116,
 86,
 83,
 108,
 108,
 16,
 23,
 32,
 61,
 107,
 74,
 14,
 74,
 129,
 133,
 36,
 8,
 80,
 16,
 46,
 92,
 82,
 1,
 80,
 80,
 28,
 80,
 107,
 72,
 9,
 82,
 123,
 8,
 108,
 111,
 6,
 41,
 122,
 83,
 52,
 47,
 10,
 129,
 14,
 118,
 108,
 80,
 107,
 90,
 73,
 32,
 107,
 49,
 73,
 120,
 38,
 4,
 58,
 58,
 21,
 38,
 121,
 128,
 43,
 73,
 81,
 58,
 82,


In [11]:
categories

{'EA01': 0,
 'EA02': 1,
 'EA03': 2,
 'EA04': 3,
 'EA05': 4,
 'EA06': 5,
 'EA07': 6,
 'EA08': 7,
 'EA09': 8,
 'EA10': 9,
 'EA11': 10,
 'EA13': 11,
 'EA14': 12,
 'EA15': 13,
 'EB01': 14,
 'EB02': 15,
 'EB03': 16,
 'EB04': 17,
 'EB05': 18,
 'EB06': 19,
 'EB07': 20,
 'EC01': 21,
 'EC02': 22,
 'EC03': 23,
 'EC04': 24,
 'EC05': 25,
 'EC06': 26,
 'EC07': 27,
 'EC09': 28,
 'ED01': 29,
 'ED02': 30,
 'ED03': 31,
 'ED04': 32,
 'ED05': 33,
 'ED06': 34,
 'ED07': 35,
 'ED08': 36,
 'ED09': 37,
 'ED10': 38,
 'ED11': 39,
 'EE01': 40,
 'EE02': 41,
 'EE03': 42,
 'EE04': 43,
 'EE05': 44,
 'EE06': 45,
 'EE07': 46,
 'EE08': 47,
 'EE09': 48,
 'EE10': 49,
 'EE11': 50,
 'EE12': 51,
 'EE13': 52,
 'EE14': 53,
 'EF01': 54,
 'EF02': 55,
 'EF04': 56,
 'EF05': 57,
 'EF06': 58,
 'EG04': 59,
 'EG06': 60,
 'EG07': 61,
 'EH01': 62,
 'EH02': 63,
 'EH03': 64,
 'EH06': 65,
 'EH07': 66,
 'EH09': 67,
 'EH10': 68,
 'EH11': 69,
 'EI01': 70,
 'EI02': 71,
 'EI03': 72,
 'EI04': 73,
 'EI05': 74,
 'EI06': 75,
 'EI08': 76,
 'EI09': 

In [12]:
idx_2_cate = {}
for i in categories:
    idx_2_cate[categories[i]] = i
    
analysis_filename = "analysis_test_{desc}.csv".format(desc=desc)
analy_x = []
analy_y = []
# for x, y in zip(str_test_x, test_y):
#     analy_x.append(x)
#     analy_y.append(idx_2_cate[y])

for x, y in zip(keywords, codes):
    analy_x.append(x)
    analy_y.append(y)

df_analy = pd.DataFrame({"input" : analy_x, "target" : analy_y})
df_analy.to_csv(analysis_filename, index=False)

In [13]:
idx_2_cate

{0: 'EA01',
 1: 'EA02',
 2: 'EA03',
 3: 'EA04',
 4: 'EA05',
 5: 'EA06',
 6: 'EA07',
 7: 'EA08',
 8: 'EA09',
 9: 'EA10',
 10: 'EA11',
 11: 'EA13',
 12: 'EA14',
 13: 'EA15',
 14: 'EB01',
 15: 'EB02',
 16: 'EB03',
 17: 'EB04',
 18: 'EB05',
 19: 'EB06',
 20: 'EB07',
 21: 'EC01',
 22: 'EC02',
 23: 'EC03',
 24: 'EC04',
 25: 'EC05',
 26: 'EC06',
 27: 'EC07',
 28: 'EC09',
 29: 'ED01',
 30: 'ED02',
 31: 'ED03',
 32: 'ED04',
 33: 'ED05',
 34: 'ED06',
 35: 'ED07',
 36: 'ED08',
 37: 'ED09',
 38: 'ED10',
 39: 'ED11',
 40: 'EE01',
 41: 'EE02',
 42: 'EE03',
 43: 'EE04',
 44: 'EE05',
 45: 'EE06',
 46: 'EE07',
 47: 'EE08',
 48: 'EE09',
 49: 'EE10',
 50: 'EE11',
 51: 'EE12',
 52: 'EE13',
 53: 'EE14',
 54: 'EF01',
 55: 'EF02',
 56: 'EF04',
 57: 'EF05',
 58: 'EF06',
 59: 'EG04',
 60: 'EG06',
 61: 'EG07',
 62: 'EH01',
 63: 'EH02',
 64: 'EH03',
 65: 'EH06',
 66: 'EH07',
 67: 'EH09',
 68: 'EH10',
 69: 'EH11',
 70: 'EI01',
 71: 'EI02',
 72: 'EI03',
 73: 'EI04',
 74: 'EI05',
 75: 'EI06',
 76: 'EI08',
 77: 'EI0

In [14]:
df_analy

,input,target
0,강관 접합부 용접 볼트 편측,EC01
1,유연신축성 생체신호 감지 유기 센서,EC02
2,크론병궤양성 대장염clusterin,LC02
3,bulk heterojunction,EF06
4,글로벌 리더 양성바이오헬스케어 소재,LA09
...,...,...
219451,산통,LB07
219452,휴폭,LB01
219453,본칩,LA08
219454,근로,EE02


In [15]:
len(max(str_train_x, key=len))

19

In [16]:
print('Loading BERT tokenizer...')
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=False)
# tokenizer = BertTokenizer.from_pretrained('models/paper_kisti_base/vocab.txt', do_lower_case=False)
tokenizer = BertTokenizer.from_pretrained('../nscc_mecab_base_final/vocab.txt', do_lower_case=False) # 최신 vocab


# MAX_LENGTH = len(max(str_train_x, key=len))
MAX_LENGTH = 25
BATCH_SIZE = 32

Loading BERT tokenizer...


/home/ubuntu/anaconda3/envs/pytorch1.4_p36/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:1646: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  FutureWarning,


In [17]:
encode_train_data = []
input_ids =[]
attention_masks =[]
token_type_ids =[]
for line in tqdm(str_train_x):
    line = ' '.join(mecab.morphs(line)) # mecab 적용
    encoded_dict = tokenizer.encode_plus(line, \
                                         add_special_tokens = True,\
                                         pad_to_max_length=True,\
                                         max_length=MAX_LENGTH, 
                                        return_attention_mask=True,
                                        truncation = True)

    input_id=encoded_dict['input_ids']
    attention_mask=encoded_dict['attention_mask']
    token_type_id = encoded_dict['token_type_ids']
    
    input_ids.append(input_id)
    attention_masks.append(attention_mask)
    token_type_ids.append(token_type_id)

train_input_ids=np.array(input_ids, dtype=int)
train_attention_masks=np.array(attention_masks, dtype=int)
train_token_type_ids=np.array(token_type_ids, dtype=int)
train_x=(train_input_ids, train_attention_masks, train_token_type_ids)

print("Original Text : ", x_data[0])
print("Tokenizer Text : ", tokenizer.tokenize(x_data[0], add_special_tokens = True, max_length=MAX_LENGTH))
print("Encode Text : ", tokenizer.encode(x_data[0], add_special_tokens = True, max_length=MAX_LENGTH))

  0%|          | 0/153619 [00:00<?, ?it/s]/home/ubuntu/anaconda3/envs/pytorch1.4_p36/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 153619/153619 [00:29<00:00, 5197.15it/s]
Keyword arguments {'add_special_tokens': True, 'max_length': 25} not recognized.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) 

Original Text :  강관  접합부  용접  볼트  편측
Tokenizer Text :  ['강', '##관', '접합부', '용접', '볼', '##트', '편', '##측']
Encode Text :  [202, 311, 3700, 13728, 8537, 1671, 3930, 3305, 3701, 203]


In [18]:
category = set(train_y)
category_idx = dict()
for n, cate in enumerate(category):
    category_idx[cate] = n
idx_train_label = []
for l in tqdm(train_y):
    idx_train_label.append(category_idx[l])
    
train_y = to_categorical(idx_train_label)

print('Train : {} / {}'.format(train_x[0].shape, train_y.shape))

100%|██████████| 153619/153619 [00:00<00:00, 1292755.36it/s]

Train : (153619, 25) / (153619, 145)


In [19]:
with tf.device('/device:GPU:2'):
    model = TFElectraClassifier(num_class=len(category))

    # 학습 준비하기
    model.compile(
    #     optimizer=keras.optimizers.Adam(learning_rate=5e-5),
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss = keras.losses.CategoricalCrossentropy(),
        metrics=['accuracy',tf.keras.metrics.TopKCategoricalAccuracy(k=3)]
    )

    checkpoint_filepath = "electra_ckpt_{desc}/ckpt".format(desc=desc)
    checkpoint_dir = os.path.dirname(checkpoint_filepath)
    model.save_weights(checkpoint_filepath.format(epoch=0))

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        monitor='val_loss',
        mode='min',
        save_best_only=True)

    model.fit(train_x, train_y, batch_size=BATCH_SIZE, validation_split=0.2, epochs=100, callbacks=[callback, model_checkpoint_callback])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFElectraModel: ['discriminator_predictions.dense.weight', 'electra.embeddings.position_ids', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing TFElectraModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFElectraModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFElectraModel for predictions without further train

Epoch 1/100
3841/3841 [==============================] - 342s 89ms/step - loss: 3.7641 - accuracy: 0.1565 - top_k_categorical_accuracy: 0.3149 - val_loss: 3.3901 - val_accuracy: 0.2128 - val_top_k_categorical_accuracy: 0.3936
Epoch 2/100
3841/3841 [==============================] - 336s 88ms/step - loss: 3.2209 - accuracy: 0.2396 - top_k_categorical_accuracy: 0.4330 - val_loss: 3.2351 - val_accuracy: 0.2428 - val_top_k_categorical_accuracy: 0.4324
Epoch 3/100
3841/3841 [==============================] - 335s 87ms/step - loss: 2.8137 - accuracy: 0.3072 - top_k_categorical_accuracy: 0.5208 - val_loss: 3.2747 - val_accuracy: 0.2513 - val_top_k_categorical_accuracy: 0.4407
Epoch 4/100
3841/3841 [==============================] - 334s 87ms/step - loss: 2.3710 - accuracy: 0.3919 - top_k_categorical_accuracy: 0.6191 - val_loss: 3.4172 - val_accuracy: 0.2478 - val_top_k_categorical_accuracy: 0.4334
Epoch 5/100
3841/3841 [==============================] - 334s 87ms/step - loss: 1.9006 - accurac

In [20]:
encode_test_data = []
input_ids =[]
attention_masks =[]
token_type_ids =[]
for line in tqdm(str_test_x):
    line = ' '.join(mecab.morphs(line)) # mecab 적용
    encoded_dict = tokenizer.encode_plus(line, \
                                         add_special_tokens = True,\
                                         pad_to_max_length=True,\
                                         max_length=MAX_LENGTH, 
                                        return_attention_mask=True,
                                        truncation = True)

    input_id=encoded_dict['input_ids']
    attention_mask=encoded_dict['attention_mask']
    token_type_id = encoded_dict['token_type_ids']
    
    input_ids.append(input_id)
    attention_masks.append(attention_mask)
    token_type_ids.append(token_type_id)

test_input_ids=np.array(input_ids, dtype=int)
test_attention_masks=np.array(attention_masks, dtype=int)
test_token_type_ids=np.array(token_type_ids, dtype=int)
test_x=(test_input_ids, test_attention_masks, test_token_type_ids)

  0%|          | 0/65837 [00:00<?, ?it/s]/home/ubuntu/anaconda3/envs/pytorch1.4_p36/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 65837/65837 [00:12<00:00, 5227.16it/s]


In [21]:
idx_test_label = []
for l in tqdm(test_y):
    idx_test_label.append(category_idx[l])
    

test_y = to_categorical(idx_test_label)

print('test : {} / {}'.format(test_x[0].shape, test_y.shape))

100%|██████████| 65837/65837 [00:00<00:00, 788866.62it/s]

test : (65837, 25) / (65837, 145)


In [22]:
with tf.device('/device:GPU:2'):
#     score_his = model.evaluate(test_x, test_y)
    y_pred = model.predict(test_x)

In [23]:
y_pred

array([[1.5952595e-02, 6.1202352e-04, 2.6994114e-04, ..., 7.7839155e-05,
        2.3667997e-04, 1.3909493e-04],
       [5.5798166e-04, 2.2607590e-03, 1.1592159e-04, ..., 6.1307219e-05,
        6.8117610e-05, 3.5665300e-05],
       [4.6492374e-04, 1.3801594e-04, 9.7584398e-06, ..., 2.7337999e-06,
        6.7190599e-06, 1.1076467e-06],
       ...,
       [5.5056752e-04, 1.4538796e-02, 2.2836434e-04, ..., 3.1853690e-06,
        1.3316093e-06, 1.3344976e-06],
       [1.5486625e-04, 3.5959908e-03, 6.3514302e-04, ..., 6.2270054e-05,
        6.5238203e-04, 8.8041008e-05],
       [1.1440460e-01, 2.0098789e-01, 4.3138256e-03, ..., 3.4987580e-02,
        4.4975782e-04, 2.6323926e-04]], dtype=float32)

In [24]:
pred_y = []
for y in y_pred:
    tmp = np.zeros(len(categories))
    tmp[y.argmax()] = 1
    pred_y.append(tmp)

In [25]:
pred_y

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.

In [26]:
y_preds = np.argsort(y_pred, axis=1)[:,-5:]

In [27]:
y_preds

array([[107, 108,  86,   5, 110],
       [ 25,  19,  37,  58,   9],
       [136,  32,   5, 110,  86],
       ...,
       [ 41,   4,   1, 110, 111],
       [107, 104,  80,  87,  88],
       [ 59,   0,  11,  10,   1]])

In [29]:
str_test_y = []
for y in tqdm(test_y):
    str_test_y.append(idx_2_cate[y.argmax()])

100%|██████████| 65837/65837 [00:00<00:00, 509061.47it/s]


In [30]:
result_top5_file_name = 'result_NSCC_top-5_{desc}.csv'.format(desc=desc)
list_preds = []
for i in range(len(y_preds)):
    inner_preds = []
    for j in range(4, -1, -1):
        inner_preds.append(idx_2_cate[y_preds[i][j]])
    list_preds.append(inner_preds)
df_result = pd.DataFrame({'inputs' : str_test_x, 'target' : str_test_y, 'preds' : list_preds})
df_result.to_csv(result_top5_file_name, index=False)

In [31]:
df_result

,inputs,target,preds
0,ECE 이미징진단,NB04,"[LC04, EA06, LA07, LC02, LC01]"
1,화학약품운반선,EA10,"[EA10, EF06, ED09, EB06, EC05]"
2,액상동작 나노공진기 기반 혈당 센서,ED04,"[LA07, LC04, EA06, ED04, NC09]"
3,노면 결빙 방지,EI04,"[EI06, EI05, EA09, EE12, EI04]"
4,난류경계층,EA07,"[ND04, ND08, ND06, EH01, ND07]"
...,...,...,...
65832,극저온 열처리,EB01,"[EB06, ED02, EF06, EB01, ED04]"
65833,티지에프베타 활성 효소,LA01,"[LA01, LC01, LA04, LC02, LA06]"
65834,첨단재활로봇,LC14,"[LC05, LC04, EA02, EA05, EE02]"
65835,카이닌산,NB05,"[LA09, LA08, LA01, LB18, LC01]"


In [32]:
top5s = []
top1s = []
for i in tqdm(range(len(df_result['target']))):
    code = df_result['target'][i]
    if code in df_result['preds'][i]:
        top5s.append(code)
    else:
        top5s.append(0)
    if code == df_result['preds'][i][0]:
        top1s.append(code)
    else:
        top1s.append(0)

100%|██████████| 65837/65837 [00:01<00:00, 56667.27it/s]


In [36]:
a = pd.DataFrame({"inputs" : df_result['inputs'], "target" : df_result["target"], "preds" : df_result["preds"], "correct" : top5s, "top1" : top1s})



In [41]:
a[a['correct'] != 0].drop(['top1'], axis=1) # 0.516

,inputs,target,preds,correct
1,화학약품운반선,EA10,"[EA10, EF06, ED09, EB06, EC05]",EA10
2,액상동작 나노공진기 기반 혈당 센서,ED04,"[LA07, LC04, EA06, ED04, NC09]",ED04
3,노면 결빙 방지,EI04,"[EI06, EI05, EA09, EE12, EI04]",EI04
7,기기 간의 통신,EE06,"[EE02, EE06, EE10, EE12, EE09]",EE06
14,설명가능한 인공지능,EE02,"[EE01, EE02, ED08, EE10, EE03]",EE02
...,...,...,...,...
65823,혈액뇌관문 투과구조,LC03,"[LA07, LC03, LC02, LC04, LC01]",LC03
65826,경상누층군,ND01,"[ND01, EF02, ND08, ND07, LA05]",ND01
65828,도시 지하매설물 모니터링,EH10,"[EI02, ND04, EH10, EH02, EH01]",EH10
65832,극저온 열처리,EB01,"[EB06, ED02, EF06, EB01, ED04]",EB01


In [40]:
a[a['top1'] != 0] # 0.249

,inputs,target,preds,correct,top1
1,화학약품운반선,EA10,"[EA10, EF06, ED09, EB06, EC05]",EA10,EA10
15,의료기기전문가,LC14,"[LC14, LC04, LC10, EG07, ED01]",LC14,LC14
16,폴리머 마이크로 장치,EA06,"[EA06, EC02, EB03, LC05, LA07]",EA06,EA06
24,공생배양 실험모델,LC01,"[LC01, LB01, LA02, LA05, LA01]",LC01,LC01
25,인듐갈륨징크 옥사이드,ED04,"[ED04, ED01, NB06, NB05, EB02]",ED04,ED04
...,...,...,...,...,...
65800,관자엽뇌전증,LC01,"[LC01, LA03, LC07, LA02, LC03]",LC01,LC01
65810,미세먼지 연변동성,ND04,"[ND04, ND06, EH01, ND08, ND05]",ND04,ND04
65819,스마트 그리드 보안,EE03,"[EE03, EE06, EE04, EE10, EE13]",EE03,EE03
65826,경상누층군,ND01,"[ND01, EF02, ND08, ND07, LA05]",ND01,ND01
